In [4]:
# Libraries for developing a Neural Network
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from keras.losses import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
sns.set_style('darkgrid')
plt.rcParams['figure.dpi'] = 108

Using TensorFlow backend.


In [7]:
dt = pd.read_csv("model_data/data.csv",sep ="\t")
y = dt.Real
test_dt =pd.read_csv("model_data/test_data.csv",sep ="\t")
ds = dt.drop('Real',axis =1)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds, y, test_size=0.2, random_state=1)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))

Training set has 6364 instances. Test set has 1591 instances.


In [9]:
poly =PolynomialFeatures(2,include_bias = False)
poly.fit(X_train)
X_train= poly.transform(X_train)
X_test= poly.transform(X_test)
test_dt = poly.transform(test_dt)

In [10]:
# standard
scaler = StandardScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
#dt = scaler.transform(dt)
test_dt = scaler.transform(test_dt)
#dt['Real'] =dt.apply(lambda x: 0 if x.Real ==-1 else 1, axis =1)


In [18]:
if __name__ == "__main__":
    #定义神经网络的参数
    w = tf.Variable(tf.random_normal([,5],stddev=1,seed=1))
    b = tf.Variable(tf.random_normal([1],stddev=1,seed=1))
    #定义输入和输出
    x = tf.placeholder(tf.float32,shape=(None,2),name="x-input")
    y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")
    #定义神经网络的前向传播过程
    y = tf.nn.sigmoid(tf.matmul(x,w) + b)
    #基于min和max对张量t进行截断操作，为了应对梯度爆发或者梯度消失的情况
    cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0))+(1-y_) * tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
    # 使用Adadelta算法作为优化函数，来保证预测值与实际值之间交叉熵最小
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    #通过随机函数生成一个模拟数据集
    rdm = RandomState(1)
    # 定义数据集的大小
    dataset_size = 100
    # 模拟输入是一个二维数组
    X = rdm.rand(dataset_size,2)
    #定义输出值，将x1+x2 < 1的输入数据定义为正样本
    Y = [[int(x1+x2 < 1)] for (x1,x2) in X]
    #创建会话运行TensorFlow程序
    with tf.Session() as sess:
        #初始化变量  tf.initialize_all_variables()
        init = tf.initialize_all_variables()
        sess.run(init)
        #设置神经网络的迭代次数
        steps = 500
        for i in range(steps):
            #通过选取样本训练神经网络并更新参数
            for (input_x,input_y) in zip(X,Y):
                input_x = np.reshape(input_x,(1,2))
                input_y= np.reshape(input_y,(1,1))
                sess.run(train_step,feed_dict={x:input_x,y_:input_y})
            #每迭代1000次输出一次日志信息
            if i % 100 == 0:
                # 计算所有数据的交叉熵
                total_cross_entropy = sess.run(cross_entropy,feed_dict={x:X,y_:Y})
                # 输出交叉熵之和
                print("After %d training step(s),cross entropy on all data is %g"%(i,total_cross_entropy))
        #预测输入X的类别
        pred_Y = sess.run(y,feed_dict={x:X})
        index = 1
        for pred,real in zip(pred_Y,Y):
            print(pred,real)

SyntaxError: invalid syntax (<ipython-input-18-787f317c7bd5>, line 3)

In [14]:
# Model arguments
args = dict(x = X_train,
            y = y_train,
            epochs=40,
            validation_split=0.2,
            verbose=2,
            shuffle=True)

# Layer definition
input_layer = Input(shape=(5,))
hidden_layer_1 = Dense(100, activation='relu',activity_regularizer=regularizers.l1(10e-5))(input_layer)
#hidden_layer_1 = Dropout(0.3)(hidden_layer_1)
#hidden_layer_2 = Dense(8, activation='sigmoid')(hidden_layer_1)
hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
#hidden_layer_2 = Dropout(0.3)(hidden_layer_2)
output_layer = Dense(2, activation='softmax')(hidden_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)

# Model set up
model.compile(tf.keras.optimizers.RMSprop(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 102       
Total params: 5,752
Trainable params: 5,752
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Execute training
model.fit(**args)

Epoch 1/40


ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
  